# Asistente Turístico de Tenerife
**Práctica Final - Módulo Large Language Models**

In [1]:
# Section 1: Setup and PDF Loading
from src.conf import OPENAI_API_KEY, MODEL_CONFIG, PDF_PATH
from src.logger import logger
from src.api_client import OpenAIClient
from src.data_loader import DataLoader

print(f"API Key loaded: {OPENAI_API_KEY[:8]}...")
print(f"Model config: {MODEL_CONFIG}")

API Key loaded: sk-proj-...
Model config: {'model': 'gpt-4o-mini', 'temperature': 0.3, 'max_tokens': 1024, 'top_p': 0.9}


In [2]:
# Initialize OpenAI client
client = OpenAIClient()

if client.test_connection():
    print("✅ OpenAI connection OK")
else:
    print("❌ OpenAI connection failed")

2026-01-14 15:56:22,049 - INFO - OpenAI client initialized with model: gpt-4o-mini
2026-01-14 15:56:22,986 - INFO - OpenAI connection test successful


✅ OpenAI connection OK


In [3]:
# Load PDF
loader = DataLoader(PDF_PATH)
pages = loader.load()
stats = loader.get_stats()
print(f"✅ PDF loaded: {stats}")

2026-01-14 15:56:31,046 - INFO - DataLoader initialized with: data/TENERIFE.pdf
2026-01-14 15:56:31,363 - INFO - Loaded 25 pages from data/TENERIFE.pdf


✅ PDF loaded: {'num_pages': 25, 'total_chars': 16094, 'total_words': 2817, 'avg_chars_per_page': 643}


In [4]:
# Preview first page content
print(loader.get_page(0)[:500])

TENERIFE – LUGARES DE INTERÉS 
SITIOS QUE VER 
 
ZONA NORTE 
 
• Santa Cruz de Tenerife: 
Santa Cruz de Tenerife es la capital de la isla. Quizás la ruta a seguir si vais a Santa 
Cruz sería: 
- Aparcar en el aparcamiento del Parque Marítimo (ubicación). 
- Caminar por la Avenida Marítima hasta Plaza de España (ubicación). 
- Por el camino de la Avenida Marítima, ver el auditorio de Tenerife (ubicación). 
- Una vez llegados a Plaza España, callejear un poco (subir la Calle Castillo 
dirección Pl
